In [ ]:
import json
import re

class TextoJson:
    def __init__(self, archivo):
        self.archivo = archivo
        self.data = {
            "antecedentes_personales": {},
            "antecedentes_familiares": {},
            "estadia_tumoral": {},
            "inmunohistoquimica_tumoral": {},
            "tipo": {},
        }
        self.keywords = [
            "edad", "sexo", "peso", "talla", "preferencia", 
            "índice tabáquico", "alcohol", "drogas", 
            "comorbilidades", "antecedentes ginecológicos"
        ]

    def preprocesar_texto(self):
        with open(self.archivo, 'r', encoding='utf-8') as file:
            lineas = file.readlines()

        texto_procesado = []
        clave_actual = None
        valor_actual = []

        for linea in lineas:
            linea = linea.strip().lower()
            if not linea:
                continue  # Ignorar líneas vacías

            #  Identificar si la línea es una nueva clave
            if re.match(r'^[a-záéíóúñ]+.*?:', linea, re.IGNORECASE) or re.match(r'^-\s*[a-záéíóúñ]+.*?', linea, re.IGNORECASE):
                # Si ya hay una clave en curso, guardar la clave y su valor
                if clave_actual:
                    if valor_actual:  # Si tiene valores asociados
                        texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")
                    else:  # Si no tiene valores, solo agregar la clave
                        texto_procesado.append(clave_actual)
                
                # Actualizar clave y reiniciar el valor
                if ':' in linea:
                    # Eliminar "-" inicial y espacios en caso de que existan
                    linea = re.sub(r'^-\s*', '', linea)
                    clave_actual = linea.split(':', 1)[0].strip()
                    valor_actual = [linea.split(':', 1)[1].strip()]
                else:
                    clave_actual = linea.strip('- ').strip()
                    valor_actual = []
            elif clave_actual:
                # Agregar la línea al valor actual
                valor_actual.append(linea)
            else:
                # Este caso es cuando la clave no tiene dos puntos
                texto_procesado.append(linea)

        # Guardar la última clave-valor si quedó algo pendiente
        if clave_actual:
            texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")

        # # Procesar enumeraciones en los valores
        # for i in range(len(texto_procesado)):
        #     texto_procesado[i] = re.sub(r'(\d+\.\s)', r'/ \1', texto_procesado[i]).strip('/ ')


        print('\n'.join(texto_procesado))
        return '\n'.join(texto_procesado)
    
    def extraer_keywords(self, lineas):
        # Crear las regex para las palabras clave
        regexes = [
            re.compile(rf'\b{re.escape(keyword)}\b:?\s*(.*)', re.IGNORECASE) 
            for keyword in self.keywords
        ]
        
        # Agregar una regex para detectar edades en el formato "XX años"
        edad_regex = re.compile(
            r'\b(?:edad:?\s*(\d+)(?:\s*años)?)|(?:\b(\d+)\s*años(?:\s*de\s*edad)?)|^(gacmc)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$',
            re.IGNORECASE
        )

        # Agregar regex para antecedentes ginecológicos
        antecedentes_ginecologicos_keywords = [
            "menarca", "fum", "trh", "estado hormonal"
        ]
        antecedentes_ginecologicos_regex = re.compile(
            rf'\b(?:{"|".join(map(re.escape, antecedentes_ginecologicos_keywords))})\b.*',
            re.IGNORECASE
        )

        antecedentes_ginecologicos_obstetrico_regex = re.compile(
            r'\b(?:g|p|c|a|o)\d+',  # Detectar combinaciones como "g0", "p1", "a2", etc.
            re.IGNORECASE
        )

        lineas = lineas.splitlines()
        resultados = {keyword: None for keyword in self.keywords}
        resultados["antecedentes_ginecológicos"] = {}
        resultados["comorbilidades"] = {}
        resultados["sexo"] = "femenino"
        edad_encontrada = False

        antecedentes_ginecologicos = []


        for linea in lineas:
            # Buscar patrones como "g0", "p0", etc.
            matches_gpa = antecedentes_ginecologicos_obstetrico_regex.findall(linea)
            if matches_gpa:
                for match in matches_gpa:
                    letra = match[0]  # La letra inicial (g, p, c, a, o)
                    valor = match[1:]  # El número que sigue (0, 1, 2, etc.)
                    resultados["antecedentes_ginecológicos"][letra] = valor  # Guardar como clave-valor
                continue

            # Buscar antecedentes ginecológicos
            match_ginecologicos = antecedentes_ginecologicos_regex.search(linea)
            

            if match_ginecologicos:
                # Dividir la línea si contiene múltiples datos separados por " / "
                partes = [parte.strip() for parte in linea.split("/") if parte.strip()]
                for parte in partes:
                    # Separar en clave-valor utilizando los dos puntos como delimitador principal
                    if ':' in parte:
                        clave_valor = parte.split(":", 1)
                    else:
                        clave_valor = re.split(r'\s+', parte, maxsplit=1)
                    
                    if len(clave_valor) == 2:
                        clave, valor = clave_valor

                        # Reemplazar espacios en la clave con guiones bajos
                        clave = clave.strip().replace(' ', '_')

                        resultados["antecedentes_ginecológicos"][clave.lower()] = valor.strip()
                    else:
                        clave = clave_valor[0].strip().replace(' ', '_')
                        resultados["antecedentes_ginecológicos"][clave.lower()] = None
                continue

            # Buscar una edad en el formato específico
            match = edad_regex.match(linea)
            if match:
                if match.group(1):  # "edad: XX" o "edad: XX años"
                    resultados["edad"] = match.group(1).strip()
                elif match.group(2):  # "XX años" o "XX años de edad"
                    resultados["edad"] = match.group(2).strip()
                elif match.group(5):  # Formato "gacmc / ... / XX años / ..."
                    resultados["edad"] = match.group(5).strip()
                edad_encontrada = True

            for i, regex in enumerate(regexes):
                match = regex.search(linea)
                if match:
                    clave = self.keywords[i]
                    valor = match.group(1).strip()
                    
                    if clave == "edad" and valor.isdigit() and not edad_encontrada: 
                        edad_encontrada = True
                        resultados["edad"] = valor
                    
                    elif clave == "comorbilidades":
                        comorbilidades = valor
                        # Dividir la cadena en comorbilidades individuales
                        comorbilidades_list = re.split(r'\d+\.\s', comorbilidades)

                        # Eliminar posibles cadenas vacías resultantes de la división
                        comorbilidades_list = [comorbilidad for comorbilidad in comorbilidades_list if comorbilidad]

                        comorbilidades_dict = {}

                        for comorbilidad in comorbilidades_list:
                            comorbilidad = comorbilidad.strip()
                            # Extraer el año
                            match = re.search(r'\((\d{4})\)', comorbilidad)
                            if match:
                                year = match.group(1)
                                # Separar nombre y descripción
                                parts = comorbilidad.split(f"({year})")
                                nombre = parts[0].strip()
                                descripcion = parts[1].strip() if len(parts) > 1 else ""
                                
                                if year not in comorbilidades_dict:
                                    comorbilidades_dict[year] = []
                                
                                comorbilidades_dict[year].append({
                                    "nombre": nombre,
                                    "descripcion": descripcion.lstrip(': ') if descripcion else None
                                })

                        # Imprimir el resultado
                        resultados[clave] = comorbilidades_dict


                        
                    elif clave != "edad":
                        resultados[clave] = valor
        # Verificar si no se encontró "índice tabáquico" y buscar "tabaco"
        if resultados.get("índice tabáquico") is None:
            for linea in lineas:
                match = re.search(r'\btabaco\b:?\s*(.*)', linea, re.IGNORECASE)
                if match:
                    resultados["índice tabáquico"] = match.group(1).strip()
                    break
                  
        ##remplazamos los espacios de las claves por guiones bajos
        resultados = {clave.replace(' ', '_'): valor for clave, valor in resultados.items()}
        return resultados

    def procesar(self):

        # Preprocesar el texto
        lineas = self.preprocesar_texto()
        # Extraer palabras clave y valores
        resultados = self.extraer_keywords(lineas)
        # Agregar los resultados a "antecedentes_personales"
        self.data["antecedentes_personales"] = resultados

        self.imprimir()

            
    def guardar_json(self, salida):
        with open(salida, 'w', encoding='utf-8') as file:
            json.dump(self.data, file, indent=4, ensure_ascii=False)

    def imprimir(self):
        print(json.dumps(self.data, indent=4, ensure_ascii=False))

# Ejemplo de uso
archivo_entrada = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\TextoExtraido2.txt"
archivo_salida = ""

procesador = TextoJson(archivo_entrada)
procesador.procesar()


{
    "antecedentes_personales": {
        "edad": "al diagnóstico) - menarca 15 años / fum 21 años - g0 p0 c0 a0 - trh: durante 30 años - estado hormonal: postmenopáusica",
        "sexo": null,
        "peso": null,
        "talla": null,
        "preferencia": null,
        "índice tabáquico": null,
        "alcohol": "ocasional",
        "drogas": "negado",
        "comorbilidades": "1. linfoma de hodgkin (1975): 3 ciclos qt (ciclofosfamida, vincristina y otro fármaco) + rt por encima del diafragma (imss) 2. cáncer papilar de tiroides (1988): tiroidectomía + iodo 131 t4/t3 3. cistoadenoma seroso microquístico de páncreas (2016) 4. cdi de mama izquierda ec ia (t1c, no, mo), rh+, her2 -: cc + gc + rt + ia x 5 años (2016)",
        "antecedentes ginecológicos": null
    },
    "antecedentes_familiares": {},
    "estadia_tumoral": {},
    "inmunohistoquimica_tumoral": {},
    "tipo": {}
}


<>:118: SyntaxWarning: invalid escape sequence '\D'
<>:118: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_18708\1086998143.py:118: SyntaxWarning: invalid escape sequence '\D'
  archivo_entrada = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\TextoExtraido2.txt"
